# Feature Engineering Part 1

On this Noteboook, the process of Feature Enginnering and Feature Selection is debugged using reduced data. After the definition of the process, all the code generated is applied to the complete data on Part 2.

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

# Feature Selection Libraries
from cnr_methods import LOFO_GPU_Importance
import xgboost as xgb

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data, y_train = get_simplified_data()
train_data = full_data[full_data['Set']=='Train']

In [3]:
train_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [4]:
WF = 'WF1'
data = train_data[train_data['WF']==WF]
y_train = y_train[y_train.index.isin(data['ID'])]

In [5]:
data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

Changing Reference for Negative Angles:

In [7]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using the original Features, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [9]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated:

In [10]:
feature_data['Month_Number'] = feature_data.index.month
feature_data['Quarter_Number'] = feature_data.index.quarter

In [11]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
median_month = feature_data.groupby('Month_Number').median()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

# Quarter
mean_quarter = feature_data.groupby('Quarter_Number').mean()[features]
median_quarter = feature_data.groupby('Quarter_Number').median()[features]
variance_quarter = feature_data.groupby('Quarter_Number').var()[features]

In [12]:
# Month
mean_month.columns = mean_month.columns + '_Month_Mean'
median_month.columns = median_month.columns + '_Month_Median'
variance_month.columns = variance_month.columns + '_Month_Variance'

# Quarter
mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

In [13]:
# Month
feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')

# Quarter
feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')

In [14]:
feature_data.index = data.index

For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [15]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [16]:
days_in_month = feature_data.index.days_in_month

In [17]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [18]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Statistics

In [19]:
for column in features:
    feature_data[column + '_Rolling_7_Window_Mean'] = feature_data[column].rolling(window=7).mean()
    feature_data[column + '_Rolling_14_Window_Mean'] = feature_data[column].rolling(window=14).mean()
    feature_data[column + '_Rolling_7_Window_Variance'] = feature_data[column].rolling(window=7).var()
    feature_data[column + '_Rolling_14_Window_Variance'] = feature_data[column].rolling(window=14).var()

### Expanding Window Statistics

In [20]:
for column in features:
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().min()
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().max()

Dropping Original Features:

In [21]:
feature_data = feature_data.drop(features,axis=1)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

A list of the generated features can be found https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html.

In [22]:
data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]

In [23]:
tsfresh_data = pd.DataFrame()
for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
    df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction: 100%|██████████| 15/15 [01:52<00:00,  7.49s/it]


Here, a little preprocess is done to convert the generated features to the appropriate tabular format.

In [24]:
tsfresh_data = tsfresh_data.pivot(columns='Feature')

In [25]:
tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)

Constant Columns are removed from data.

In [26]:
tsfresh_data = tsfresh_data.loc[:, tsfresh_data.apply(pd.Series.nunique) != 1]

In [27]:
tsfresh_data.head()

,value__abs_energy|CLCT,value__abs_energy|T,value__abs_energy|U_100m,value__abs_energy|U_10m,value__abs_energy|V_100m,value__abs_energy|V_10m,value__absolute_sum_of_changes|CLCT,value__absolute_sum_of_changes|T,value__absolute_sum_of_changes|U_100m,value__absolute_sum_of_changes|U_10m,...,value__variance|U_100m,value__variance|U_10m,value__variance|V_100m,value__variance|V_10m,value__variance_larger_than_standard_deviation|CLCT,value__variance_larger_than_standard_deviation|T,value__variance_larger_than_standard_deviation|U_100m,value__variance_larger_than_standard_deviation|U_10m,value__variance_larger_than_standard_deviation|V_100m,value__variance_larger_than_standard_deviation|V_10m
id,,,,,,,,,,,,,,,,,,,,,
2018-05-01 02:00:00,6813.370572,82047.873600,5.055752,1.574030,10.613261,0.083919,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 03:00:00,16811.539518,163992.661200,10.982542,7.778652,12.704466,0.254793,17.447701,0.180,0.186000,1.236305,...,0.008649,0.382112,0.820564,0.003824,1.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 04:00:00,26487.652359,246117.891825,12.472335,8.772845,12.775686,2.257409,19.071310,0.495,1.399929,2.730120,...,0.284984,0.425138,1.513166,0.253942,1.0,0.0,0.0,0.0,1.0,0.0
2018-05-01 05:00:00,35486.186521,327217.540225,26.210477,9.248391,51.431749,3.181777,22.577941,2.290,6.327000,3.037614,...,6.250924,0.467797,5.034489,0.202684,1.0,0.0,1.0,0.0,1.0,0.0
2018-05-01 06:00:00,44684.124127,408135.031825,40.752497,9.333069,81.075418,3.268781,23.623216,2.610,6.433900,3.436218,...,8.045999,0.556415,5.149121,0.198239,1.0,0.0,1.0,0.0,1.0,0.0


## Feature Selection

Finally, all the Features Generated have to be filtered, so only the most relevant ones are passed to the model, as a way of avoiding Overfitting. For this process, a process called LOFO (Leave One Feature Out) is selected.

The process is simple: For each Feature, a arbitrary Model (Here a XGBoost) is cross validated on a dataset that contains all the features except one, which importance is being measured, and the precision without that feature is compared to a baseline, where all features are present. The features whose removal leads to worse results are considered most important features to the dataset.

More info about this process can be found on https://github.com/aerdem4/lofo-importance.

The implementation of this method on this however, was an adaptation of the original code to allow use of GPU resources as a faster way to obtain results, since this work involves a big number of columns.

In [31]:
final_features = feature_data.merge(tsfresh_data,left_on=feature_data.index,right_on=tsfresh_data.index,how='left')
final_features = final_features.rename({'key_0':'Date'},axis=1)

In [33]:
features = final_features.drop(['ID','WF','Set','Date'],axis=1).columns

In [37]:
param = {'tree_method' : 'gpu_hist'}

In [38]:
importance_df = LOFO_GPU_Importance(final_features,y_train,features,param)

1/106 2.764100 s/it
2/106 3.569551 s/it
3/106 2.992104 s/it
4/106 3.442222 s/it
5/106 2.601523 s/it
6/106 3.57915 s/it
7/106 3.128969 s/it
8/106 3.189122 s/it
9/106 2.956525 s/it
10/106 3.133022 s/it
11/106 3.934001 s/it
12/106 3.603064 s/it
13/106 3.725968 s/it
14/106 3.230000 s/it
15/106 3.992024 s/it
16/106 4.71023 s/it
17/106 3.887030 s/it
18/106 3.424997 s/it
19/106 3.704078 s/it
20/106 3.358157 s/it
21/106 4.192590 s/it
22/106 3.750518 s/it
23/106 3.828540 s/it
24/106 3.543998 s/it
25/106 3.579537 s/it
26/106 3.256514 s/it
27/106 3.507887 s/it
28/106 3.240510 s/it
29/106 3.361510 s/it
30/106 3.398531 s/it
31/106 3.503510 s/it
32/106 3.298000 s/it
33/106 3.460006 s/it
34/106 3.411000 s/it
35/106 3.299999 s/it
36/106 3.166011 s/it
37/106 3.353002 s/it
38/106 3.275553 s/it
39/106 3.421763 s/it
40/106 3.338512 s/it
41/106 3.370018 s/it
42/106 3.681573 s/it
43/106 3.78513 s/it
44/106 3.205026 s/it
45/106 3.174019 s/it
46/106 3.222999 s/it
47/106 3.173868 s/it
48/106 3.154001 s/it
49/1

In [39]:
importance_df.to_csv(r'C:\Users\andre_\OneDrive\Documentos\Feature Selection\Importance_WF1.csv')

,feature,score
2179,value__time_reversal_asymmetry_statistic__lag_...,-0.720504
246,"value__agg_linear_trend__f_agg_""var""__chunk_le...",-0.669485
906,"value__cwt_coefficients__widths_(2, 5, 10, 20)...",-0.605652
1101,"value__cwt_coefficients__widths_(2, 5, 10, 20)...",-0.513798
1554,value__friedrich_coefficients__m_3__r_30__coef...,-0.414610
919,"value__cwt_coefficients__widths_(2, 5, 10, 20)...",-0.408635
1491,"value__fft_coefficient__coeff_8__attr_""real""|CLCT",-0.404268
434,value__c3__lag_3|V_100m,-0.368895
1227,value__energy_ratio_by_chunks__num_segments_10...,-0.361866
1135,"value__cwt_coefficients__widths_(2, 5, 10, 20)...",-0.343983
